In [ ]:
# silver_patients_scd1_batch.py
# SOURCE: kardia_bronze.bronze_patients (Delta CDF)
# OUTPUT: `kardia_silver.silver_patients`, updated incrementally.
# PATTERN: Read Change Data Feed from Bronze, mask PHI columns, derive `BIRTH_YEAR` from `BIRTHDATE`.
# TRIGGER: Incremental batch job

from kflow.config import CHANGE_TYPES, PHI_COLS_MASK, silver_paths, bronze_table

from delta.tables import DeltaTable
from pyspark.sql import functions as F, Window

# Load Silver paths
S = silver_paths("patients")
SRC_TABLE = bronze_table("patients")
TGT_TABLE = S.table

In [ ]:
# 1. Ensure Silver Patients table exists
spark.sql(
    f"""
    CREATE TABLE IF NOT EXISTS {TGT_TABLE} (
        id           STRING  NOT NULL,
        birth_year   INT,
        deathdate    STRING,
        first        STRING,
        last         STRING,
        ssn          STRING,
        drivers      STRING,
        passport     STRING,
        birthplace   STRING,
        marital      STRING,
        race         STRING,
        ethnicity    STRING,
        gender       STRING
    ) USING DELTA
    """
)

In [ ]:
# 2. Merge the latest version of each patient record into the Silver Patients table.
#    `batch_df` is a static DF containing the latest new and updated rows from Bronze CDF.
#    Use the `_commit_version` column from CDF to identify the most recent change per ID.
def upsert_to_silver(batch_df, _):
    w_latest = Window.partitionBy("ID").orderBy(F.col("_commit_version").desc())

    latest = (
        batch_df
          .filter(F.col("_change_type").isin(*CHANGE_TYPES))
          .filter(F.col("ID").isNotNull())
          .withColumn("rn", F.row_number().over(w_latest))
          .filter("rn = 1")
          .select(
              F.col("ID").alias("id"),
              F.year("BIRTHDATE").alias("birth_year"),
              *[F.lit(None).cast("string").alias(c.lower()) for c in PHI_COLS_MASK],
              F.col("MARITAL").alias("marital"),
              F.col("RACE").alias("race"),
              F.col("ETHNICITY").alias("ethnicity"),
              F.col("GENDER").alias("gender")
          )
    )

    (DeltaTable.forName(spark, TGT_TABLE)
               .alias("t")
               .merge(latest.alias("s"), "t.id = s.id")
               .whenMatchedUpdateAll()
               .whenNotMatchedInsertAll()
               .execute())

In [ ]:
# 3. Incremental batch run
(spark.readStream
      .format("delta")
      .option("readChangeFeed", "true")
      .table(SRC_TABLE)

      .writeStream
      .foreachBatch(upsert_to_silver)
      .option("checkpointLocation", S.checkpoint)
      .trigger(availableNow=True)
      .start()
      .awaitTermination())

In [ ]:
# 4. Batch finished – Verify Silver Patients table row count and checkpoint path.
df = spark.table(TGT_TABLE)
print(f"Silver Patients row count: {df.count():,}")
display(df.limit(5))

# NOTE: A Delta Lake merge is not a join in the relational sense.
# It is a mutation command where the source table drives the operation.
#
# Delta CDF emits all changes since the last checkpoint.
# This may include multiple versions of the same ID across different commits.
# We deduplicate using row_number() to keep only the latest version per patient ID.